## Using the EWXPWSDB StationReadings Class

# <span style="color:red">clear all output before saving: db output contains passwords! </span>

this walks through process of

- creating a temporary DB
- using the StationReading class for existing station record to
 - get station info
 - mess around with timezones
 - pull readings by date interval
 - pull hourly and daily summaries of readings

In [1]:
%load_ext autoreload
%autoreload 2

## Constants/Config

In [2]:

station_file = '../data/test_stations.tsv'
station_type = 'SPECTRUM'
station_code = 'EWXSPECTRUM01' 

In [3]:

from ewxpwsdb.db.database import create_temp_pg_engine, get_db_url, init_db, Session
from ewxpwsdb.db.models import WeatherStation, Reading, StationType, APIResponse


## create engine temp database

In [4]:

engine = create_temp_pg_engine(get_db_url(), name_prefix = 'notebook_testing')

temp_db_url = engine.url
print(temp_db_url.database)
init_db(engine,station_file)


notebook_testing_mh9lz7da9n
Station with type 'DAVIS' merged into the database
Station with type 'LOCOMOS' merged into the database
Station with type 'ONSET' merged into the database
Station with type 'RAINWISE' merged into the database
Station with type 'SPECTRUM' merged into the database
Station with type 'ZENTRA' merged into the database


In [5]:
from sqlalchemy import Engine, text, inspect

with Session(engine) as session:
    results = session.exec(text('show timezone;')).all()
    print(results)
    
    

[('UTC',)]


In [7]:
from ewxpwsdb.station import Station

station = Station.from_station_code(station_code, engine)
print(station.weather_station.station_type)
print('does our station match the station type we expected?')
station.weather_station.station_type == station_type


SPECTRUM
does our station match the station type we expected?


True

## Get some data into our temp database

In [8]:
from ewxpwsdb.collector import Collector

collector = Collector(station.weather_station, engine)
print(collector.station_code)

EWXSPECTRUM01


set up a time interval to pull data for

In [9]:
from datetime import datetime, UTC, timedelta
from ewxpwsdb.time_intervals import UTCInterval

today_utc = datetime.now(UTC).date()

response_ids = collector.request_and_store_weather_data_utc(UTCInterval.one_day_interval())
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")

somerex = collector.request_and_store_weather_data_utc(UTCInterval.one_day_interval(d = today_utc- timedelta(days = 1)))
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")




store 48 readings for EWXSPECTRUM01
store 287 readings for EWXSPECTRUM01


In [ ]:
# 
response_ids = collector.catch_up()
n_readings = len(collector.current_reading_ids)
print(f"store {n_readings} readings for {station_code}")

## StationReadings class

In [10]:
from ewxpwsdb.station import Station
from sqlmodel import select

with Session(engine) as session:            
    stmt = select(WeatherStation).where(WeatherStation.station_code == station_code)
    station_record:WeatherStation= session.exec(stmt).one()

station_record

WeatherStation(station_code='EWXSPECTRUM01', id=3, lat=42.70736, location_description='EWX Field Office', api_config='{"sn":"50400123","apikey":"11a5c3a939856b08677b7a072f8e6865"}', install_date=datetime.date(2023, 5, 1), timezone='US/Eastern', station_type='SPECTRUM', ewx_user_id='0', lon=84.46512, background_place='HTC')

In [11]:
station_record.model_dump_json()

'{"station_code":"EWXSPECTRUM01","id":3,"lat":42.70736,"location_description":"EWX Field Office","api_config":"**api login hidden**","install_date":"2023-05-01","timezone":"US/Eastern","station_type":"SPECTRUM","ewx_user_id":"0","lon":84.46512,"background_place":"HTC"}'

In [12]:
test_station = Station.from_station_code(station_code, engine)
test_station.weather_station


WeatherStation(station_code='EWXSPECTRUM01', id=3, lat=42.70736, location_description='EWX Field Office', api_config='{"sn":"50400123","apikey":"11a5c3a939856b08677b7a072f8e6865"}', install_date=datetime.date(2023, 5, 1), timezone='US/Eastern', station_type='SPECTRUM', ewx_user_id='0', lon=84.46512, background_place='HTC')

In [13]:
test_station.weather_station.id

3

In [14]:
sql = test_station.weatherstation_plus_sql()
for line in sql.split('\n'):
    print(line)

select 
            weatherstation.*,
            stationtype.sampling_interval,
            stationtype.supported_variables,
            min(reading.data_datetime at time zone weatherstation.timezone) first_reading_datetime, 
            min(reading.data_datetime) as first_reading_datetime_utc,
            max(reading.data_datetime at time zone weatherstation.timezone) as latest_reading_datetime, 
            max(reading.data_datetime) as latest_reading_datetime_utc,
            60/stationtype.sampling_interval::int as expected_readings_hour,
            (24*60)/stationtype.sampling_interval::int as expected_readings_day
        from 
            weatherstation inner join stationtype on weatherstation.station_type = stationtype.station_type
            inner join reading on weatherstation.id = reading.weatherstation_id 
        where 
            weatherstation.id = 3
        group by 
            weatherstation.id, stationtype.sampling_interval, stationtype.supported_variables
      

In [15]:
test_station.station_with_detail(engine).model_dump()

{'id': 3,
 'station_code': 'EWXSPECTRUM01',
 'station_type': 'SPECTRUM',
 'install_date': datetime.date(2023, 5, 1),
 'timezone': 'US/Eastern',
 'ewx_user_id': '0',
 'lat': 42.70736,
 'lon': 84.46512,
 'location_description': 'EWX Field Office',
 'background_place': 'HTC',
 'api_config': SecretStr('**********'),
 'sampling_interval': 5,
 'expected_readings_day': 288,
 'expected_readings_hour': 12,
 'first_reading_datetime': datetime.datetime(2024, 6, 24, 20, 0),
 'first_reading_datetime_utc': datetime.datetime(2024, 6, 25, 0, 0, tzinfo=datetime.timezone.utc),
 'latest_reading_datetime': datetime.datetime(2024, 6, 25, 23, 55),
 'latest_reading_datetime_utc': datetime.datetime(2024, 6, 26, 3, 55, tzinfo=datetime.timezone.utc),
 'supported_variables': '["atmp", "lws", "pcpn", "relh", "srad", "wspd", "wdir", "wspd_max"]'}

## Debugging WeatherStationDetail class

this is a class that is Pydantic Base model with class methods to create from station code and engine

In [18]:
from ewxpwsdb.station import WeatherStationDetail
wsd:WeatherStationDetail = WeatherStationDetail.with_detail(station_code = station_code, engine = engine)
print(wsd.model_dump_json(indent = 4))
wsd.first_reading_datetime

{
    "id": 3,
    "station_code": "EWXSPECTRUM01",
    "station_type": "SPECTRUM",
    "install_date": "2023-05-01",
    "timezone": "US/Eastern",
    "ewx_user_id": "0",
    "lat": 42.70736,
    "lon": 84.46512,
    "location_description": "EWX Field Office",
    "background_place": "HTC",
    "api_config": "**********",
    "sampling_interval": 5,
    "expected_readings_day": 288,
    "expected_readings_hour": 12,
    "first_reading_datetime": "2024-06-24T20:00:00",
    "first_reading_datetime_utc": "2024-06-25T00:00:00Z",
    "latest_reading_datetime": "2024-06-25T23:55:00",
    "latest_reading_datetime_utc": "2024-06-26T03:55:00Z",
    "supported_variables": "[\"atmp\", \"lws\", \"pcpn\", \"relh\", \"srad\", \"wspd\", \"wdir\", \"wspd_max\"]"
}


datetime.datetime(2024, 6, 24, 20, 0)

In [27]:
assert wsd.first_reading_datetime is not None

In [22]:
# can we use supported vars to check things?  that field looks it's stored kinda wonky in data model, but it's JSON!

var_of_interest = 'lws'
import json
if var_of_interest in json.loads(wsd.supported_variables):
    print('var is in there -- we can do the thing!')

var is in there -- we can do the thing!


In [ ]:
from ewxpwsdb.station_readings import StationReadings


station_readings = StationReadings(station = test_station.weather_station, engine = engine)

print(station_readings.station.station_code)
print(station_readings.station.id)


In [ ]:
readings = station_readings.recent_readings()
print(readings)

create a date interval to use to pull data

In [ ]:

from zoneinfo import ZoneInfo
from datetime import date
from ewxpwsdb.time_intervals import DateInterval

dates = DateInterval(start = date(2024, 6, 8), end = date(2024, 6, 12), local_timezone = ZoneInfo("America/Detroit") )
utc_interval = dates.to_utc_datetime_interval()


Get more readings with collector, pull them with station_readings

In [ ]:
collector.request_and_store_weather_data_utc(utc_interval)
print(f"stored {len(collector.current_reading_ids)} readings ")



Testing complex SQL to pull hourly summary of readings by date

In [ ]:
from ewxpwsdb.db.summary_models import HourlySummary

start_date=dates.start
end_date=dates.end
sql_str = HourlySummary.sql_str(station_id= station.weather_station.id, local_start_date=dates.start, local_end_date=dates.end, station_timezone=station.weather_station.timezone)
for line in sql_str.split("\n"):
    print(line)

In [ ]:
one_day = DateInterval(start = date(2024, 6, 9), end = date(2024, 6, 10), local_timezone = ZoneInfo("America/Detroit") )

records = station_readings.hourly_summary(local_start_date = one_day.start, local_end_date=one_day.end )
print(len(records))
if (len(records) > 0 ):
    print(records[0])
    


### Clean up

remove test database

In [ ]:

from ewxpwsdb.db.database import drop_pg_db, list_pg_databases
from sqlalchemy.orm import close_all_sessions

# if collector:
#     collector._session.close()
#     collector._engine.dispose()

close_all_sessions()

print(f"attempting to drop db {engine.url.database}")
result = drop_pg_db(engine)
print(result)
engine.dispose()
list_pg_databases(host='localhost')

